In [38]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [39]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        #1 input chanel, 6 output channels, 5x5 square convolution
        #kernel(
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2= nn.Conv2d(6,16,5)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x =F.max_pool2d(F.relu(self.conv1(x)), (2,2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x =self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x. size()[1:] #all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [40]:
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [41]:
params = list(net.parameters())
print(len(params))
print(params[0].size())

10
torch.Size([6, 1, 5, 5])


try random input

In [42]:
input =torch.randn(1,1,32,32) #if you have a single sample, use input.unsqueeze(0) to add a fake batch dimension
out = net(input)
print(out)
out.size()

tensor([[ 0.0685,  0.1218, -0.0247,  0.0035, -0.0791, -0.0868,  0.0955, -0.0726,
          0.0057,  0.1449]], grad_fn=<ThAddmmBackward>)


torch.Size([1, 10])

In [43]:
net.zero_grad()

In [44]:
out.backward(torch.randn(1,10))

Loss function

In [45]:
output = net(input)
target = torch.randn(10) # dummy target, for example
target = target.view(1,-1) # same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(0.3759, grad_fn=<MseLossBackward>)


Backprop

In [46]:
net.zero_grad()
print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])


In [47]:
loss.backward()

In [48]:
print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad after backward
tensor([ 0.0001, -0.0140,  0.0048,  0.0035,  0.0081, -0.0118])


Update the weights

In [49]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sata ub_(f.grad.d* learning_rate)

In [50]:
import torch.optim as optim

In [52]:
#chose optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

#in the training loop
optimizer.zero_grad() #zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()